Tratando de analizar por qué AdamW está dando error

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# Definir un modelo simple con una capa lineal
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear = nn.Linear(10, 1)  # Mapea un tensor de tamaño 10 a un escalar

    def forward(self, x):
        return self.linear(x)

# Crear el modelo y el optimizador
model = SimpleModel()
optimizer = optim.AdamW(model.parameters(), lr=0.01)

# Crear datos aleatorios de entrada y etiquetas
inputs = torch.randn(32, 10)  # Batch de 32, cada muestra de tamaño 10
targets = torch.randn(32, 1)  # Etiquetas correspondientes de tamaño 32x1

# Definir una función de pérdida (MSE en este caso)
criterion = nn.MSELoss()

# Forward pass
outputs = model(inputs)
loss = criterion(outputs, targets)

# Backward pass
optimizer.zero_grad()  # Resetea los gradientes de los parámetros
loss.backward()        # Calcula los gradientes con respecto a la pérdida
optimizer.step()       # Actualiza los parámetros del modelo

print("Training step completed successfully.")

Training step completed successfully.
